# 선형 회귀(Linear Regression)

선형으로 나타나는 데이터에서 보편적인 방정식을 찾아내 모델(함수)를 만드는 것. 

y = Wx + b에서 W와 b를 찾는것. (손실함수와 최적화를 통해)

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [3]:
torch.manual_seed(1)

In [6]:
x_train = torch.FloatTensor([[1], [2], [3]]) 
y_train = torch.FloatTensor([[2], [4], [6]]) 

이 둘의 관계를 나타내는 함수를 정의하고 싶은 것이다. 즉, 최종 목적은 **일부의 데이터**로 일반화하여 어떤 x가 들어오더라도 y값을 예측할 수 있도록 하는 것이다.

In [7]:
print(x_train)
print(x_train.shape)

tensor([[1.],
        [2.],
        [3.]])
torch.Size([3, 1])


In [8]:
print(y_train)
print(y_train.shape)

tensor([[2.],
        [4.],
        [6.]])
torch.Size([3, 1])


In [9]:
# 가중치를 0으로 초기화
W = torch.zeros(1, requires_grad=True) # requires_grad -> 학습을 통해 값이 변경되는 텐서임을 알려줌.

In [11]:
print(W)

tensor([0.], requires_grad=True)


In [12]:
# 편향을 0으로 초기화
b = torch.zeros(1, requires_grad=True)
print(b)

tensor([0.], requires_grad=True)


현재 직선의 방정식은 
y = 0*x + 0 임

In [13]:
# hypothesis : 가설 = 직선의 방정식
hypothesis = x_train * W + b
print(hypothesis)

tensor([[0.],
        [0.],
        [0.]], grad_fn=<AddBackward0>)


가중치(기울기)와 편향(절편)이 모두 0이니  당연히 모든 y값도 0.

In [14]:
# 비용 함수 "평균 제곱 오차" 선언
cost = torch.mean((hypothesis - y_train) ** 2)
print(cost)

tensor(18.6667, grad_fn=<MeanBackward0>)


In [15]:
# 확률적 경사 하강법 <- 옵티마이저
optimizer = optim.SGD([W, b], lr=0.01)

여기서 lr은 learning rate의 약자로 학습률을 의미한다. 

새로운 가중치는 기존의 가중치에서 (학습률*손실함수를 w에 대해서 미분한 값)을 뺀 값으로 갱신된다. 따라서 학습률이 높으면 높을수록 가중치가 크게크게 갱신된다.

In [18]:
# 훈련

nb_epochs = 1000

for epoch in range(1, nb_epochs+1):
  # 예측
  hypothesis = x_train * W + b

  # 손실 계산
  cost = torch.mean((hypothesis - y_train) ** 2)

  # 가중치 갱신
  optimizer.zero_grad() # 그레디언트 초기화
  cost.backward() # 역전파 연산 수행 
  optimizer.step()

  # 100 에포크마다 로그 출력
  if epoch % 100 == 0:
    print('Epoch {:4d}/{} W: {:.3f}, b: {:.3f} Cost: {:.6f}'.format(epoch, nb_epochs, W.item(), b.item(), cost.item()))


Epoch  100/1000 W: 1.842, b: 0.358 Cost: 0.018483
Epoch  200/1000 W: 1.876, b: 0.281 Cost: 0.011421
Epoch  300/1000 W: 1.903, b: 0.221 Cost: 0.007058
Epoch  400/1000 W: 1.923, b: 0.174 Cost: 0.004361
Epoch  500/1000 W: 1.940, b: 0.137 Cost: 0.002695
Epoch  600/1000 W: 1.953, b: 0.107 Cost: 0.001665
Epoch  700/1000 W: 1.963, b: 0.084 Cost: 0.001029
Epoch  800/1000 W: 1.971, b: 0.066 Cost: 0.000636
Epoch  900/1000 W: 1.977, b: 0.052 Cost: 0.000393
Epoch 1000/1000 W: 1.982, b: 0.041 Cost: 0.000243


신기하지 않은가? 

비용이 점점 감소한다는 뜻은 예측과 정답이 거의 비슷해지고 있다는 것을 의미한다. 즉, 모델은 정답을 더 잘 맞추기 위해서 가중치와 절편을 점점 수정해나간다.


## optimizer.zero_grad()가 필요한 이유

매 에포크마다 optimizer.zero_grad()를(그레디언트 초기화) 호출하는 이유는 무엇일까?

파이토치는 미분을 통해 얻은 기울기를 이전에 계산된 기울기 값에 누적시키는 특징이 있기 때문이다.

In [20]:
import torch
w = torch.tensor(2.0, requires_grad=True)

nb_epochs = 20
for epoch in range(1, nb_epochs + 1):
  z = 2*w
  z.backward()
  print('수식을 w로 미분한 값 : {}'.format(w.grad))

수식을 w로 미분한 값 : 2.0
수식을 w로 미분한 값 : 4.0
수식을 w로 미분한 값 : 6.0
수식을 w로 미분한 값 : 8.0
수식을 w로 미분한 값 : 10.0
수식을 w로 미분한 값 : 12.0
수식을 w로 미분한 값 : 14.0
수식을 w로 미분한 값 : 16.0
수식을 w로 미분한 값 : 18.0
수식을 w로 미분한 값 : 20.0
수식을 w로 미분한 값 : 22.0
수식을 w로 미분한 값 : 24.0
수식을 w로 미분한 값 : 26.0
수식을 w로 미분한 값 : 28.0
수식을 w로 미분한 값 : 30.0
수식을 w로 미분한 값 : 32.0
수식을 w로 미분한 값 : 34.0
수식을 w로 미분한 값 : 36.0
수식을 w로 미분한 값 : 38.0
수식을 w로 미분한 값 : 40.0


z를 w로 미분하면 2가 나온다. 그런데 그 값이 계속해서 더해지고(누적되고) 있다. 이렇게 되면 에포크를 반복할 때마다 누적된 그레디언트 값이 역전파 되므로 모델이 수렴하지 않을 것이다. 따라서 매 에포크마다 zero_grad()로 초기화를 해줘야 한다.

## 자동 미분(Autograd)

앞서 텐서에다가 required_grad를 True로 지정해줬다. 이는 텐서에 대한 기울기(미분값)을 저장하겠다는 의미이다. 

w.grad 에 w에 대해 미분한 값이 저장된다.

In [22]:
import torch

w = torch.tensor(2.0, requires_grad=True)

y = w**2
z = 2*y + 5

z.backward() # 역전파 연산에서 미분이 되고 미분값이 저장된다.

print(print('수식을 w로 미분한 값 : {}'.format(w.grad)))

수식을 w로 미분한 값 : 8.0
None


즉 함수 z에 대하여 w=2일 때의 접선의 기울기 8이 저장된다는 뜻이다.